In [30]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [31]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,280,043 tx:0x9cf9e575e38a535ed06bcc077639251a24e9de32c34988ce6ed9ad722a0e9167
Swap at block:60,280,043 tx:0x4ac5bdb920fa00e8e3916d2409f64b88f7597b3b3518c273731c049a926cafc9
Swap at block:60,280,044 tx:0xdc2e6389a4238bb3c8b643ecbfb00281f5317fc4bb2cfaf432d7ac3cc013bb97
Swap at block:60,280,044 tx:0x668908b8200731c9a83755932d53f950389e9e0c7e18c8a24f8ee998b5fdbe82
Swap at block:60,280,046 tx:0x572dcc234b9caf60c965413297485e55d8f54a86479883cd75e0f0cfdc784875
Swap at block:60,280,046 tx:0x572dcc234b9caf60c965413297485e55d8f54a86479883cd75e0f0cfdc784875
Swap at block:60,280,048 tx:0xce4aa08c80735dd5330e1ae4c2e90b0aa4c8cd321a558f8104cb99bfee76faa7
Swap at block:60,280,051 tx:0x8b21a688a9b00e81c8d0a610e7a07bcfaf75630a8eba61d255c5d67a9623864d
.


In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0xb18ec753acd9ecbfdff996a5eaec3bbab823868f,0x9cf9e575e38a535ed06bcc077639251a24e9de32c349...,60280043,1722988004,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0xb18ec753acd9ecbfdff996a5eaec3bbab823868f,0x4ac5bdb920fa00e8e3916d2409f64b88f7597b3b3518...,60280043,1722988004,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x369582d2010b6ed950b571f4101e3bb9b554876f,0xdc2e6389a4238bb3c8b643ecbfb00281f5317fc4bb2c...,60280044,1722988006,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0xdef834ae4a1198bfec84544fb374ec7f1314a7c5,0x668908b8200731c9a83755932d53f950389e9e0c7e18...,60280044,1722988006,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0xa789324e64268c5385ea7678435fa83532705b0f,0x572dcc234b9caf60c965413297485e55d8f54a864798...,60280046,1722988010,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0x7ccc755dbe531af53657742fc00df3bef39b5fa6,0x572dcc234b9caf60c965413297485e55d8f54a864798...,60280046,1722988010,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0xa030be97a53d6462c675962fec3eafbe53b8bb6c,0xce4aa08c80735dd5330e1ae4c2e90b0aa4c8cd321a55...,60280048,1722988016,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0x232eb60e4ea68a164988329e16305bcacccf2a97,0x8b21a688a9b00e81c8d0a610e7a07bcfaf75630a8eba...,60280051,1722988022,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xb18ec753acd9ecbfdff996a5eaec3bbab823868f',
  'tx_hash': '0x9cf9e575e38a535ed06bcc077639251a24e9de32c34988ce6ed9ad722a0e9167',
  'blk_num': 60280043,
  'timestamp': 1722988004,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0x6A7264186Df6a12b62997BE14100bADD9062Af47',
   'amount0In': 367893000000000000,
   'amount1Out': 58794029447415139889942}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xb18ec753acd9ecbfdff996a5eaec3bbab823868f',
  'tx_hash': '0x4ac5bdb920fa00e8e3916d2409f64b88f7597b3b3518c273731c049a926cafc9',
  'blk_num': 60280043,
  'timestamp': 1722988004,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x00000000041d945c46E073F0048cEf510D148dEA',
   'token1': '0x00000000041d945c46E073F0048cEf510D148dE